# Ocean Heat Content - regrid
To ease the computational load, I regridded the yearly averaged temperature and potential density files from the original curvilinear 0.1 degree grid to the a rectangular 0.4 degree grid.

This notebook compared the OHC behaviour of the `ctrl` and `lpd` runs.

In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot
matplotlib.rc_file('../rc_file')

In [ ]:
from paths import path_samoc, path_results
from scipy.optimize import curve_fit
from xr_regression import xr_quadtrend

In [ ]:
ctrl = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl.nc')
lpd  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_lpd.nc' )
spinup = 50  # how many years to ignore due to spinup effects: data from year 51 of ctrl run

f, ax = plt.subplots(1, 2 , figsize=(8,3), sharey=True, gridspec_kw={"width_ratios":[len(ctrl.time), len(lpd.time)]})
for i in range(2):
    ax[i].axhline(0, c='grey', lw=.5)
    ax[i].set_xlabel('time [model years]')
    
for i, ocean in enumerate(['Global', 'Atlantic', 'Pacific', 'Southern']):
    key = f'OHC_{ocean}_Ocean'
    c = ['k' ,'C0','C1','C3'][i]
    ax[0].plot(ctrl.time/365, (ctrl[key]-ctrl[key][0])/1e21, c=c)
    ax[1].plot(lpd .time/365, (lpd[key]-lpd[key][0]  )/1e21, c=c , ls='--', label=f'{ocean} Ocean')

ax[0].set_ylabel('OHC anomaly [ZJ]')
    
ax[0].axvspan(0, spinup, alpha=0.3, color='grey')
ax[1].axvspan(lpd.time[0+300-spinup]/365, lpd.time[-1]/365, alpha=0.3, color='grey', label='discarded data')

ax[1].legend(frameon=False)

ctrl = ctrl.isel(time=np.arange(50,300))
ctrl = ctrl.assign_coords(time=ctrl.time.values/365)
lpd  = lpd .isel(time=np.arange(0,300-spinup))
lpd  = lpd .assign_coords(time=lpd .time.values/365)
plt.savefig(f'{path_results}/OHC/OHC_anomalies_ctrl_lpd')

grey shaded areas indicated discarded data

## equilibration time

In [ ]:
def adjustment_time(da, plot=True):
    def exp_decay(x, a, b):
        return a*np.exp(-x/b)
    
    Delta_da = (da - da.shift(time=1))
    popt, pcov = curve_fit(exp_decay, Delta_da.time[1:], Delta_da.values[1:], p0=(Delta_da.values[1], 1000.))
    tau = popt[1]

    if plot==True:
        cax = plt.gca()
        cax.axhline(0, c='k', lw=.5)
        cax.plot(Delta_da.time[1:], Delta_da.values[1:]/1e21)
        cax.plot(Delta_da.time[1:], exp_decay(Delta_da.time[1:], *popt)/1e21, 'r-', label="Fitted Curve")
        tau_symbol = r'$\tau$'
        cax.text(.1, .85, f'{tau_symbol} = {tau:3.0f} years', transform=cax.transAxes, fontsize=16)
        cax.set_ylim((-10,20))
    return tau

In [ ]:
# time series: depth-lat-lon integrated OHC
fig = plt.figure(figsize=(8,8))
ax = []
for i, ocean in enumerate(['Global', 'Atlantic', 'Pacific', 'Southern']):
    ax1 = fig.add_subplot(4,2,2*i+1)
    ax.append(ax1)
    Delta = r'$\Delta$'
    ax1.set_ylabel(f'{Delta}OHC {ocean} [ZJ]')
    adjustment_time(ctrl[f'OHC_{ocean}_Ocean'])
    
    ax2 = fig.add_subplot(4,2,2*i+2)
    ax2.set_yticklabels([])
    ax.append(ax2)
    adjustment_time(lpd [f'OHC_{ocean}_Ocean'])
for i in range(3):
    j = 2*i
    ax[-1].get_shared_x_axes().join(ax[-1], ax[j+2])
    ax[-2].get_shared_x_axes().join(ax[-2], ax[j+3])
    ax[j].set_xticklabels([])
    ax[j+1].set_xticklabels([])
for i in range(2):
    ax[-i-1].set_xlabel('model years')

plt.subplots_adjust(bottom=0.06, left=.08, right=0.99, top=0.99, wspace=.1, hspace=.1)
# plt.savefig(f'{path_results}/OHC/OHC_equilibration_full')  # comment out time selection above
plt.savefig(f'{path_results}/OHC/OHC_equilibration_select')


In [ ]:
# z-profiles: lat-lon integrated OHC


## Latitudinal patterns

In [ ]:
lpd_lat = lpd.TLAT.mean(axis=1)

In [ ]:
f, ax = plt.subplots(1, 2 , figsize=(8,3))
for i, ocean in enumerate(['Global', 'Atlantic', 'Pacific', 'Southern']):
    key = f'OHC_zonal_{ocean}_Ocean'
    c = ['k' ,'C0','C1','C3'][i]
# mean
    ax[0].plot(ctrl.t_lat, ctrl[key].mean(dim='time')    , c=c , label=ocean)
    ax[0].plot(lpd_lat   , lpd [key].mean(dim='time')/100, c=c , ls='--')

# std of quad. detrended
    ax[1].plot(ctrl.t_lat, (ctrl[key]-xr_quadtrend(ctrl[key])).std(dim='time'), c=c)
    ax[1].plot(lpd_lat   , (lpd[key] -xr_quadtrend(lpd[key] )).std(dim='time')/100, c=c , ls='--')
ax[0].legend(frameon=False)

plt.savefig(f'{path_results}/OHC/OHC_zonal_mean_std_ctrl_lpd')

In [ ]:
extents = [(-78,90), (-40,80), (-40,70), (-40,30)]
height_ratios = [a[1]-a[0] for a in extents]
f, ax = plt.subplots(4, 3 , figsize=(8,10), sharex='col', gridspec_kw={"width_ratios":[1,1, 0.05], "height_ratios":height_ratios})
cY, cX = np.meshgrid(ctrl.t_lat, ctrl.time)
lY, lX = np.meshgrid(lpd.TLAT.mean(axis=1), lpd.time)
vex, ims = [3e16, 2e16, 2e16, 1e16], []
for i, ocean in enumerate(['Global', 'Atlantic', 'Pacific', 'Indian']):
    kwargs = {'cmap':'RdBu', 'vmin':-vex[i], 'vmax':vex[i]}
    key = f'OHC_zonal_{ocean}_Ocean'
    im = ax[i,0].pcolormesh(cX, cY, ctrl[key] -xr_quadtrend(ctrl[key]), **kwargs)
    ims.append(im)
    ax[i,1].pcolormesh(lX, lY, (lpd[key]-xr_quadtrend(lpd[key]))/100, **kwargs)
    for j in range(2):  
        ax[i,j].axhline(0, c='grey', lw=.5, ls='--')
        ax[i,j].set_yticks(np.arange(-60,100,30))
        ax[i,j].set_ylim(extents[i])
    if i==0:
        ax[0,0].text(.05, .2, 'Southern Ocean', c='g', transform=ax[0,0].transAxes)
        for j in range(2):
            ax[0,j].axhline(-31.5, c='g', lw=.8)
            ax[0,j].text(.5, 1.02, ['CTRL', 'LPD'][j], transform=ax[0,j].transAxes)
            ax[-1,j].set_xlabel('time [model years]')
    ax[i,0].text(.05, .9, ocean, c='g', transform=ax[i,0].transAxes)
    ax[i,0].set_ylabel('latitude')
    ax[i,0].get_shared_y_axes().join(ax[i,0], ax[i,1])
    cb = f.colorbar(ims[i], cax=ax[i,2], ticks=np.arange(-3e16,4e16,1e16))
    cb.outline.set_visible(False)
plt.savefig(f'{path_results}/OHC/OHC_zonal_Hovmoeller_ctrl_lpd')